In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('data/housing.csv')

In [ ]:
data = pd.get_dummies(data)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.drop(['SalePrice'], axis=1), 
                                                    data.SalePrice, test_size=0.3, 
                                                    random_state=0)

# Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFECV

# use any other model you want here.
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=411)

# build the RFE with CV option.
rfe = RFECV(model, min_features_to_select = 3, step = 1 , cv=5, scoring='accuracy')

In [ ]:
# fit the RFE to our data.
selection  = rfe.fit(x_train, y_train)

# print the selected features.
print(x_train.columns[selection.support_]) 

# Recursive Feature Addition

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error 

# array to hold the feature to be keept.
features_to_keep = [x_train.columns[0]]

# set this value according to you.
threshold = 0.002

# create your prefered model and  fit it to the training data.
model_one_feature = RandomForestRegressor(n_estimators=332)
model_one_feature.fit(x_train[[x_train.columns[0]]], y_train)

# evaluate against your metric.
y_pred_test = model_one_feature.predict(x_test[[x_train.columns[0]]])
mse_score_all = mean_squared_error(y_test, y_pred_test)

In [ ]:
# start iterating from the feature.
for feature in x_train.columns[1:]:
    
    model = RandomForestRegressor(n_estimators=332)
    
    # fit model with  the selected features and the feature to be evaluated
    model.fit(x_train[features_to_keep + [feature]], y_train)
    y_pred_test = model.predict(x_test[features_to_keep + [feature]])
    mse_score_int = mean_squared_error(y_test, y_pred_test)

    # determine the drop in the roc-auc
    diff_auc = mse_score_int - mse_score_all

    # compare the drop in roc-auc with the threshold
    if diff_auc >= threshold:
        
        # if the increase in the roc is bigger than the threshold
        # we keep the feature and re-adjust the roc-auc to the new value
        # considering the added feature
        mse_score_all = amse_score_int
        features_to_keep.append(feature)

In [ ]:
# print the feature to keep.
print(features_to_keep)